In [53]:
import numpy as np
import pandas as pd

class KdNode(object):
    def __init__(self,pInfo=None,pSplit=None,pLeft=None,pRight=None):

        self.leftChild = pLeft
        self.rightChild = pRight
        self.info = pInfo
        self.splitFeature = pSplit

def createKdTree(pDataset):
    '''
    :param pDataset:训练样本数据
    :return: KD-Tree
    :function 创建KNN二叉树，理论上应该将样本数据存放在叶子结点上
    这里将数据存储在所有节点上（包括叶子节点和非叶子节点）
    '''
    if len(pDataset) <= 0:
        return

    if len(pDataset) == 1:
        node = KdNode()
        node.info = pDataset.iloc[0]
        node.splitFeature = -1
        return node

    dataset = pd.DataFrame(pDataset)

    max_var = -9999999
    split_feature = -1
    for ix, col in dataset.iteritems():
        std = col.std()
        if(std > max_var):
            max_var = std
            split_feature = ix

    if split_feature >= 0:
        dataset.sort_values(by=split_feature,axis=0,inplace=True)
    else:
        return

    split_point = dataset.iloc[int(dataset.shape[0]/2)]
    node = KdNode(split_point,split_feature)
    node.leftChild = createKdTree(dataset.iloc[0:int(dataset.shape[0]/2)])
    node.rightChild = createKdTree(dataset.iloc[int(dataset.shape[0]/2+1):])

    return node

def printTree(pRoot):
    '''
    :param pRoot: KD-Tree根节点
    :function 打印KD-Tree
    '''
    if(pRoot):
        print(pRoot.info.values)
    else:
        return

    printTree(pRoot.leftChild)
    printTree(pRoot.rightChild)

def predict(pRoot,pQuery):
    '''
    :param pRoot:KD-Tree根节点
    :param pQuery: 查找的节点
    :return: 和pQuery距离最近的节点和最近距离（欧氏距离）
    '''
    pQuery = pd.Series(pQuery)
    cur_node = pRoot
    nearest_node = cur_node
    min_distance = 999999999
    node_stack = []

    while cur_node:
        node_stack.append(cur_node)
        cur_distance = np.sum((pQuery - cur_node.info)**2)
        if cur_distance < min_distance:
            min_distance = cur_distance
            nearest_node = cur_node

        split_feature = cur_node.splitFeature
        if split_feature >= 0:
            if(pQuery[split_feature] <= cur_node.info[split_feature]):
                cur_node = cur_node.leftChild
            else:
                cur_node = cur_node.rightChild
        else:
            break

    while node_stack:
        back_point = node_stack.pop()
        split_feature = back_point.splitFeature

        cur_distance = np.sqrt(np.sum((pQuery - back_point.info) ** 2))
        if min_distance > cur_distance:
            min_distance = cur_distance
            nearest_node = back_point

        if(split_feature >= 0):
            temp_node = None
            #if( np.sum((pQuery - back_point.info)**2) < min_distance):
            # 这个地方用欧氏距离是错误的，不会得到最小值
            if( np.abs(pQuery[split_feature] - back_point.info[split_feature]) < min_distance ):
                if pQuery[split_feature] <= back_point.info[split_feature]:
                    temp_node = back_point.rightChild
                else:
                    temp_node = back_point.leftChild
            if temp_node:
                node_stack.append(temp_node)
                cur_distance = np.sqrt(np.sum((pQuery-temp_node.info)**2))
                if min_distance > cur_distance:
                    min_distance = cur_distance
                    nearest_node = temp_node

    return nearest_node, min_distance

x = np.array(
        [
        [2.0,2.1,2.2],
        [2.2,2.4,2.1],
        [1.9,2.0,3.0],
        [1.0,1.1,1.5],
        [1.0,1.0,1.8],
        [0.0,0.0,0.5],
        [0.1,0.0,0.3],
        [0.0,0.1,0.5],
        [0.4,0.5,0.6],
        [0.5,0.4,0.4],
        [1.0,2.1,2.2],
        [2.2,2.4,2.1],
        [3.9,3.0,3.0],
        [4.0,3.1,1.5],
        [5.0,4.0,1.8],
        [6.0,5.0,0.5],
        [7.1,0.0,0.3],
        [8.0,0.1,1.5],
        [9.4,1.5,0.6],
        [10.5,1.4,0.4],
        ]
    )

def calcDistance(px,py):
    '''
    :param px:训练样本数据
    :param py: 查询样本
    :return: 查询样本和所有样本的距离（欧氏距离），有小到大排序
    :function 主要用于验证KD-Tree输出结果
    '''
    y = np.tile(py,(px.shape[0],1))
    x = (px - py)**2
    x = np.sqrt(np.sum(x,axis=1))
    x.sort(axis=0)
    return x

In [54]:
if __name__ == "__main__":
    #创建训练数据
    for i in np.arange(3):
        x = np.hstack((x,np.random.random((20, 1))))
    root = createKdTree(x)

    #待查询的数据
    y = [0.0,0.0,0.0,1.0,2.0,3.0]

    #y和x的距离
    d = calcDistance(x,y)
    print(d)

    #查询和y距离最近的样本
    a,b = predict(root,y)
    print(a.info,b)

[ 2.61189069  2.82640815  2.94807576  3.02999578  3.20872809  3.51097047
  3.70178796  4.40734235  4.7278331   4.76592463  4.85448176  5.22430016
  6.12928649  6.44364522  7.18379693  7.60113665  8.58846703  8.76535646
  9.90699347 10.99613047]
0    0.000000
1    0.100000
2    0.500000
3    0.259699
4    0.925890
5    0.795411
Name: 7, dtype: float64 2.611890689584745


In [33]:
import cv2
import numpy as np
from PIL import Image
import requests
from io import BytesIO
# import matplotlib
import os
# matplotlib.use('TkAgg')
# import matplotlib.pyplot as plt

def dHash(img):
    # 差值哈希算法
    # 缩放8*8
    img = cv2.resize(img, (9, 8))
    # 转换灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hash_str = ''
    # 每行前一个像素大于后一个像素为1，相反为0，生成哈希
    for i in range(8):
        for j in range(8):
            if gray[i, j] > gray[i, j+1]:
                hash_str = hash_str+'1'
            else:
                hash_str = hash_str+'0'
    return hash_str

def cmpHash(hash1, hash2):
    # Hash值对比
    # 算法中1和0顺序组合起来的即是图片的指纹hash。顺序不固定，但是比较的时候必须是相同的顺序。
    # 对比两幅图的指纹，计算汉明距离，即两个64位的hash值有多少是不一样的，不同的位数越小，图片越相似
    # 汉明距离：一组二进制数据变成另一组数据所需要的步骤，可以衡量两图的差异，汉明距离越小，则相似度越高。汉明距离为0，即两张图片完全一样
    n = 0
    # hash长度不同则返回-1代表传参出错
    if len(hash1) != len(hash2):
        return -1
    # 遍历判断
    for i in range(len(hash1)):
        # 不相等则n计数+1，n最终为相似度
        if hash1[i] != hash2[i]:
            n = n + 1
    return n
 
def getImageByUrl(url):
    # 根据图片url 获取图片对象
    html = requests.get(url, verify=False)
    image = Image.open(BytesIO(html.content))
    return image

def runAllImageSimilaryFun(para1, para2):
    # 均值、差值、感知哈希算法三种算法值越小，则越相似,相同图片值为0
    # 三直方图算法和单通道的直方图 0-1之间，值越大，越相似。 相同图片为1
 
    # t1,t2   14;19;10;  0.70;0.75
    # t1,t3   39 33 18   0.58 0.49
    # s1,s2  7 23 11     0.83 0.86  挺相似的图片
    # c1,c2  11 29 17    0.30 0.31
 
    if para1.startswith("http"):
         # 根据链接下载图片，并转换为opencv格式
        img1 = getImageByUrl(para1)
        img1 = cv2.cvtColor(np.asarray(img1), cv2.COLOR_RGB2BGR)
 
        img2 = getImageByUrl(para2)
        img2 = cv2.cvtColor(np.asarray(img2), cv2.COLOR_RGB2BGR)
    else:
        # 通过imread方法直接读取物理路径
        img1 = cv2.imread(para1)
        img2 = cv2.imread(para2)
 
 
    hash1 = dHash(img1)
    hash2 = dHash(img2)
    n2 = cmpHash(hash1, hash2)
#     return n2
    print('差值哈希算法相似度dHash：', n2)

In [34]:
path = "D:/image/dataset/image_test1/hg/"
file_name = os.listdir(path)

item_list =[] 
for item in file_name:
    if item[-4:].lower() == '.jpg' or item[-4:].lower() == '.png' or item[-5:].lower() == '.jpeg':
        item = path + item
        item_list.append(item)

        
list_set = []
for i in range(len(item_list)):
    for j in range(i+1,len(item_list)):
        list = [item_list[i],item_list[j]]
        list_set.append(list)      

if __name__ == "__main__":
    for i in range(len(list_set)):
        p1= list_set[i][0]
        p2= list_set[i][1]
        n2 = runAllImageSimilaryFun(p1,p2)
#         if n2 <= 10:
#             print(p1,p2)

差值哈希算法相似度dHash： 35
差值哈希算法相似度dHash： 31
差值哈希算法相似度dHash： 32
差值哈希算法相似度dHash： 29
差值哈希算法相似度dHash： 32
差值哈希算法相似度dHash： 29
差值哈希算法相似度dHash： 33
差值哈希算法相似度dHash： 28
差值哈希算法相似度dHash： 33
差值哈希算法相似度dHash： 41
差值哈希算法相似度dHash： 36
差值哈希算法相似度dHash： 34
差值哈希算法相似度dHash： 37
差值哈希算法相似度dHash： 26
差值哈希算法相似度dHash： 16
差值哈希算法相似度dHash： 19
差值哈希算法相似度dHash： 20
差值哈希算法相似度dHash： 23
差值哈希算法相似度dHash： 24
差值哈希算法相似度dHash： 30
差值哈希算法相似度dHash： 21
差值哈希算法相似度dHash： 42
差值哈希算法相似度dHash： 34
差值哈希算法相似度dHash： 39
差值哈希算法相似度dHash： 35
差值哈希算法相似度dHash： 24
差值哈希算法相似度dHash： 19
差值哈希算法相似度dHash： 9
差值哈希算法相似度dHash： 22
差值哈希算法相似度dHash： 21
差值哈希算法相似度dHash： 24
差值哈希算法相似度dHash： 28
差值哈希算法相似度dHash： 21
差值哈希算法相似度dHash： 44
差值哈希算法相似度dHash： 38
差值哈希算法相似度dHash： 39
差值哈希算法相似度dHash： 31
差值哈希算法相似度dHash： 18
差值哈希算法相似度dHash： 15
差值哈希算法相似度dHash： 23
差值哈希算法相似度dHash： 22
差值哈希算法相似度dHash： 19
差值哈希算法相似度dHash： 33
差值哈希算法相似度dHash： 20
差值哈希算法相似度dHash： 43
差值哈希算法相似度dHash： 41
差值哈希算法相似度dHash： 42
差值哈希算法相似度dHash： 30
差值哈希算法相似度dHash： 17
差值哈希算法相似度dHash： 14
差值哈希算法相似度dHash： 25
差值哈希算法相似度dHash： 24
差值哈希算法相似度dHas

In [2]:
import cv2
import numpy as np
from PIL import Image
import requests
from io import BytesIO
# import matplotlib
import os
def dHash(img):
    # 差值哈希算法
    # 缩放8*8
    img = cv2.resize(img, (9, 8))
    # 转换灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hash_str = ''
    # 每行前一个像素大于后一个像素为1，相反为0，生成哈希
    for i in range(8):
        for j in range(8):
            if gray[i, j] > gray[i, j+1]:
                hash_str = hash_str+'1'
            else:
                hash_str = hash_str+'0'
    return hash_str

def runAllImageSimilaryFun(para1):
    # 均值、差值、感知哈希算法三种算法值越小，则越相似,相同图片值为0
    # 三直方图算法和单通道的直方图 0-1之间，值越大，越相似。 相同图片为1
 
    # t1,t2   14;19;10;  0.70;0.75
    # t1,t3   39 33 18   0.58 0.49
    # s1,s2  7 23 11     0.83 0.86  挺相似的图片
    # c1,c2  11 29 17    0.30 0.31
 
    if para1.startswith("http"):
         # 根据链接下载图片，并转换为opencv格式
        img1 = getImageByUrl(para1)
        img1 = cv2.cvtColor(np.asarray(img1), cv2.COLOR_RGB2BGR)
    else:
        # 通过imread方法直接读取物理路径
        img1 = cv2.imread(para1) 
        
    hash = dHash(img1)
    return hash

path = "D:/image/dataset/image_test1/hg/"
file_name = os.listdir(path)

item_list =[] 
for item in file_name:
    if item[-4:].lower() == '.jpg' or item[-4:].lower() == '.png' or item[-5:].lower() == '.jpeg':
        item = path + item
        item_list.append(item)
item_list

       
# list_set = []
# for i in range(len(item_list)):
#     for j in range(i+1,len(item_list)):
#         list = [item_list[i],item_list[j]]
#         list_set.append(list)      
nn = []
if __name__ == "__main__":
    for i in range(len(item_list)):
        p = item_list[i]
        n = runAllImageSimilaryFun(p)
        nn.append(n)

In [3]:
nn

['0000111101100101000101110100101100000111000011110100111101000010',
 '0000111100001111000101110000101000110111010011111000111000101011',
 '0000111100001111000111110001101100110111000011111010011110101111',
 '0010111100101101001011110100110100010011100100110000111000110110',
 '0011001110001111101010011010000001100100001011010101010010000111',
 '1001011100101011110110111001101100100111101001110100111000110011',
 '0000101100011011000011111001011100001110000100110010111101001111',
 '0000000000000000000000000101001010110101110001100000000000000000',
 '1011111000110110001001101011010100111011101111100011100000001011',
 '0000111100001111001011111010111101100101001111110001111110000000',
 '0111001001011000101000101111000111010000000100001100000010011000',
 '0101000111110010101100000100011011000001010011001110000011100000',
 '0011100110100000101101000110001011111000110100001110000011100000',
 '0011001110001111101010011010000001100100001011010101010010000111',
 '00010111010010110001110100101011

In [4]:
lista = []
for i in range(len(nn)):
    a = []
    for j in range(len(nn[i])):
        b = int(nn[i][j])
        a.append(b)    
    lista.append(a)
lista

[[0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0],
 [0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1],
 [0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1],
 [0,
  0,
  1,
  0,
  1,
  1,
  1,
  

In [5]:
y = lista[0]

In [6]:
x = lista[1:]
# x = np.array(x)
# x.shape

In [7]:
import numpy as np
import pandas as pd

class KdNode(object):
    def __init__(self,pInfo=None,pSplit=None,pLeft=None,pRight=None):

        self.leftChild = pLeft
        self.rightChild = pRight
        self.info = pInfo
        self.splitFeature = pSplit

def createKdTree(pDataset):
    '''
    :param pDataset:训练样本数据
    :return: KD-Tree
    :function 创建KNN二叉树，理论上应该将样本数据存放在叶子结点上
    这里将数据存储在所有节点上（包括叶子节点和非叶子节点）
    '''
    if len(pDataset) <= 0:
        return

    if len(pDataset) == 1:
        node = KdNode()
        node.info = pDataset.iloc[0]
        node.splitFeature = -1
        return node

    dataset = pd.DataFrame(pDataset)

    max_var = -9999999
    split_feature = -1
    for ix, col in dataset.iteritems():
        std = col.std()
        if(std > max_var):
            max_var = std
            split_feature = ix

    if split_feature >= 0:
        dataset.sort_values(by=split_feature,axis=0,inplace=True)
    else:
        return

    split_point = dataset.iloc[int(dataset.shape[0]/2)]
    node = KdNode(split_point,split_feature)
    node.leftChild = createKdTree(dataset.iloc[0:int(dataset.shape[0]/2)])
    node.rightChild = createKdTree(dataset.iloc[int(dataset.shape[0]/2+1):])

    return node

def printTree(pRoot):
    '''
    :param pRoot: KD-Tree根节点
    :function 打印KD-Tree
    '''
    if(pRoot):
        print(pRoot.info.values)
    else:
        return

    printTree(pRoot.leftChild)
    printTree(pRoot.rightChild)

def predict(pRoot,pQuery):
    '''
    :param pRoot:KD-Tree根节点
    :param pQuery: 查找的节点
    :return: 和pQuery距离最近的节点和最近距离（欧氏距离）
    '''
    pQuery = pd.Series(pQuery)
    cur_node = pRoot
    nearest_node = cur_node
    min_distance = 999999999
    node_stack = []

    while cur_node:
        node_stack.append(cur_node)
        cur_distance = np.sum((pQuery - cur_node.info)**2)
        if cur_distance < min_distance:
            min_distance = cur_distance
            nearest_node = cur_node

        split_feature = cur_node.splitFeature
        if split_feature >= 0:
            if(pQuery[split_feature] <= cur_node.info[split_feature]):
                cur_node = cur_node.leftChild
            else:
                cur_node = cur_node.rightChild
        else:
            break

    while node_stack:
        back_point = node_stack.pop()
        split_feature = back_point.splitFeature

        cur_distance = np.sqrt(np.sum((pQuery - back_point.info) ** 2))
        if min_distance > cur_distance:
            min_distance = cur_distance
            nearest_node = back_point

        if(split_feature >= 0):
            temp_node = None
            #if( np.sum((pQuery - back_point.info)**2) < min_distance):
            # 这个地方用欧氏距离是错误的，不会得到最小值
            if( np.abs(pQuery[split_feature] - back_point.info[split_feature]) < min_distance ):
                if pQuery[split_feature] <= back_point.info[split_feature]:
                    temp_node = back_point.rightChild
                else:
                    temp_node = back_point.leftChild
            if temp_node:
                node_stack.append(temp_node)
                cur_distance = np.sqrt(np.sum((pQuery-temp_node.info)**2))
                if min_distance > cur_distance:
                    min_distance = cur_distance
                    nearest_node = temp_node

    return nearest_node, min_distance

x = np.array(x)

def calcDistance(px,py):
    '''
    :param px:训练样本数据
    :param py: 查询样本
    :return: 查询样本和所有样本的距离（欧氏距离），有小到大排序
    :function 主要用于验证KD-Tree输出结果
    '''
    y = np.tile(py,(px.shape[0],1))
    x = (px - py)**2
    x = np.sqrt(np.sum(x,axis=1))
#     x.sort(axis=0)
    return x

In [8]:
if __name__ == "__main__":
    #创建训练数据
    d = calcDistance(x,y)
    print(d)
    
    for i in np.arange(64):
        x = np.hstack((x,np.random.random((15, 1))))
    root = createKdTree(x)

    #待查询的数据
    y = y

    #y和x的距离
    
    #查询和y距离最近的样本
    a,b = predict(root,y)
    print(a.info,b)

[4.         4.35889894 4.47213595 5.91607978 4.79583152 4.89897949
 5.47722558 5.91607978 4.58257569 6.4807407  5.83095189 6.244998
 5.91607978 4.89897949 4.35889894]
0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      1.000000
         ...   
123    0.061665
124    0.629931
125    0.712650
126    0.924301
127    0.008867
Name: 0, Length: 128, dtype: float64 4.0
